In [4]:
from pysat.formula import CNF
from pysat.solvers import *
import re
import pandas as pd
import os

CONFIG_DIR = "../linux-4.14.152"
DIMACS_FLA = 'examples/out.dimacs'
CONFIG = "../linux-4.14.152/.config"
CSV = 'examples/alloptions-x86.4.14.152.csv'
CONF_FLAG = ["randconfig"] # , "defconfig", "allnoconfig", "allyesconfig", "allmodconfig", "alldefconfig"]

In [5]:
variables = {}
with open(DIMACS_FLA, 'r') as f:
    dimacs_lines = f.readlines()
    for line in dimacs_lines:
        if line.startswith("c"):
            m = re.search('(c) (\d+) (\w+)', line)
            var_id = int(m.group(2))
            var_name = m.group(3)  
            variables[var_id] = var_name

def search_kconfig_type(option_name, df_options):
    ktype = df_options.query("option == " + '"' + option_name + '"')['type']
    return ktype.values[0]

# we are thinking the DIMACS id (number k) corresponding to the "module" variable of an option
def search_kmodule(option_name):
    for k, v in variables.items():
        if v == option_name + "_MODULE":
            return k

def get_dict(filename):
    """Dictionary from a linux config file"""
    res = dict()
    with open(CONFIG, 'r') as f:
        config_lines = f.readlines()
        for line in config_lines:
            if not line.startswith("#") and line != '\n':
                m = re.search('CONFIG_(\w+)=([\w"-/]+)', line)
                var_name = m.group(1)
                var_val = m.group(2)
                res[var_name] = var_val
    return res

def get_var(option):
    for k, v in variables.items():
        if v == option:
            return k
        
def tristate_repartition(config_dict):
    return {'yes' : list(config_dict.values()).count('y'),
            'mod' : list(config_dict.values()).count('m')}

In [6]:
def get_valuation(config_dict):
    assumptions = []
    df_options = pd.read_csv(CSV)
    for k, v in variables.items():
        if v in config_dict:
            if (v.endswith("_MODULE") or "CHOICE_" in v):  
                continue # ignore
            kconfig_type = search_kconfig_type(v, df_options)
            if (kconfig_type == 'TRISTATE'):
                # we have to find the other "module" variable that emulates the "m" value 
                kmodule = search_kmodule(v)
                # YES
                if config_dict[v] == 'y':
                    assumptions.extend([k, -kmodule])
                # MODULE
                elif config_dict[v] == 'm':
                    assumptions.extend([-k, kmodule])
            elif (kconfig_type == 'BOOL'):
                # we have to find the other "module" variable that emulates the "m" value 
                kmodule = search_kmodule(v)
                assumptions.extend([k])
    for f in df_options.option:
        # Feature in the CSV and in the DIMACS but not in the config
        if f not in config and f in variables.values():
            k = get_var(f)
            kconfig_type = search_kconfig_type(f, df_options)
            if (kconfig_type == 'TRISTATE'):
                kmodule = search_kmodule(f)
                # NO
                assumptions.extend([-k, -kmodule])
            elif (kconfig_type == 'BOOL'):
                kmodule = search_kmodule(f)
                # NO
                assumptions.extend([-k])
    return assumptions

In [12]:
config = get_dict(CONFIG)
fla = CNF(DIMACS_FLA)
literals = get_valuation(config)
with Solver(bootstrap_with=fla.clauses) as l:
    l.set_phases(literals=literals)
    if l.solve():
        print("SAT")
    else:
        print("UNSAT")

SAT
